# Data preprocessing & EDA

## Train data(new radiator)

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

file_path = '/your_path/Data Sheet'

excel_file = pd.ExcelFile(file_path)
sheet_names = excel_file.sheet_names
print("Sheet names:", sheet_names)

sheet_name = sheet_names[0]
train = pd.read_excel(file_path, sheet_name=sheet_name)
train

### Drop those colums we are not using

In [84]:

columns_to_drop = ["Product ID", "Name", "Family", "Material"]
train = train.drop(columns=columns_to_drop)

In [ ]:
print(train.dtypes)

In [ ]:
import matplotlib.pyplot as plt

# number of unique values in each column
uniqueValues = train.nunique()

plt.figure(figsize=(10, 6))
ax = uniqueValues.plot(kind='bar', color='skyblue')
for patch in ax.patches: # count on top of each bar
    height= patch.get_height()
    width=patch.get_width()
    ax.text(patch.get_x() + width / 2,height,
            f'{int(patch.get_height())}', ha='center', va='bottom')

plt.title('Number of unique values in each column of New Radiators')
plt.xlabel('Columns')
plt.ylabel('Unique value count')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
#unique values in categorical columsn for new radiators
print("Unique values in categorical columns:")
for col in train.select_dtypes(include=['object']).columns:
    print(train[col].value_counts())
    print('\n')

In [ ]:
# value counts for 'Nipple size'
import seaborn as sns
plt.figure(figsize=(6, 5))
sns.countplot(x=train['Nipple size'], order=train['Nipple size'].value_counts().index)
plt.title("Value counts of Nipple Size (New Radiators)")
plt.xlabel('Nipple size')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

### Missing data

In [ ]:
missing_values = train.isnull().sum()
plt.figure(figsize=(5, 5))
missing_values.plot(kind='bar')
plt.title('Number of Missing Values in Each Column of the train DataFrame')
plt.xlabel('Columns')
plt.ylabel('Number of Missing Values in New Radiators')
plt.xticks(rotation=90)
plt.show()

In [90]:
# Missing data
train['Mid section Depth'] = train.apply(lambda row: row['Leg section depth'] if row['Mid section Depth'] == 'Missing data' else row['Mid section Depth'], axis=1)
train['Surface area'] = train['Surface area'].replace('Missing data', 0.29)

### Convert every column into float

In [91]:
# convert to float
train['Surface area'] = train['Surface area'].astype(float)
train['Nipple size'] = train['Nipple size'].str.replace('"', '').astype(float)

In [ ]:
print(train.dtypes)

### Convert mm to dm to ensures consistency and prevents unit mismatches

In [93]:
# convert to decimeters
train['Section length'] = train['Section length'] / 100
train['Leg section depth'] = train['Leg section depth'] / 100
train['Mid section Depth'] = train['Mid section Depth'] / 100
train['Leg section height'] = train['Leg section height'] / 100
train['Mid section height'] = train['Mid section height'] / 100
train['Nipple size'] = train['Nipple size'] * 25.4 / 100

In [ ]:
train.head()

In [ ]:
#heatmap shwoing correaltion
plt.figure(figsize=(10, 6))
sns.heatmap(train.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix for New Radiators")
plt.show()

In [ ]:
#boxplots

#we make three sets of boxplots since the distributions are significantly different
#this leads to better underatnding and visaulisations

colList1 = ['Leg section depth', 'Mid section Depth', 'Output ΔT50']

# Plot boxplots for 'Leg section depth', 'Mid section Depth', 'Output ΔT50'
plt.figure(figsize=(6, 6))
sns.boxplot(data=train[colList1], palette='coolwarm')
plt.title('Boxplots of Leg Section Depth, Mid Section Depth, and OutputΔ50')
plt.xlabel('Columns')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.show()

colList2 = ['Leg section height', 'Mid section height', 'Interaxis distance']
#boxplots for 'Leg section height', 'Mid section height', 'Interaxis distance'
plt.figure(figsize=(6, 6))
sns.boxplot(data=train[colList2], palette='coolwarm')
plt.title('Boxplots of Leg Section Height, Mid Section Height, and Inter Axis Distance')
plt.xlabel('Columns')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.show()

#boxplots for all other numeric columns
plt.figure(figsize=(12, 8))
toDrop=colList2 + colList1
train.drop(columns=toDrop).boxplot()
plt.suptitle("Boxplots of other numeric columns")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# pairplot for numeric columsn in train
import seaborn as sns
import numpy as np
numericCols = train.select_dtypes(include=[np.number])
sns.pairplot(numericCols)
plt.suptitle('Pair Plots for All Numeric Columns in Tarin DataFrame')
plt.show()

In [ ]:
numplots = len(numericCols.columns)
cols = 3
rows=np.ceil(numplots / cols)
rows = int(rows)
fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))  # Create a grid
axes = axes.flatten()

for i, col in enumerate(numericCols.columns):
    sns.histplot(numericCols[col], kde=True, ax=axes[i], bins=20)
    axes[i].set_title(f'Histogram and KDE of {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')

for j in range(i + 1, len(axes)):  # hiding unused subplots
    axes[j].axis('off')

plt.tight_layout()
plt.suptitle('Histograms and KDE for Numeric Columns in New Radiators')
plt.show()

In [ ]:
medianValues = numericCols.median()
modeValues = numericCols.mode().iloc[0]  # we take first mode if there are multiple
print("Median Values for Each Column:")
print(medianValues)
print('\n')
print("Mode Values for Each Column:")
print(modeValues)

In [ ]:
meanValues = numericCols.mean()  #calculating means

plt.figure(figsize=(10, 6))
ax = meanValues.plot(kind='bar', color='skyblue')

for patch in ax.patches: #value labels on top of each bar
    height = patch.get_height()
    width=patch.get_width()
    ax.text(patch.get_x() + width/ 2, height + 0.05, f'{height:.2f}',
            ha='center', va='bottom')
plt.title('Mean of numeric columns in New Radiators')
plt.xlabel('Columns')
plt.ylabel('Mean Value')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Test data (old radiator)

In [ ]:
import pandas as pd

file_path = 'Manchester University Data Sheet - Mar 2025.xlsx'

excel_file = pd.ExcelFile(file_path)
sheet_names = excel_file.sheet_names
print("Sheet names:", sheet_names)

sheet_name = sheet_names[1]
test = pd.read_excel(file_path, sheet_name=sheet_name)
test

In [ ]:
test.describe()

In [ ]:
test.info()

In [ ]:

# plot fro value counts of the 'Data confidence' column
dataConfidenceCounts = test['Data confidence'].value_counts()

# Plot the value counts
plt.figure(figsize=(6, 5))
dataConfidenceCounts.plot(kind='bar')

for i, value in enumerate(dataConfidenceCounts):
    plt.text(i, value + 0.01, str(value), ha='center', va='bottom')

plt.title('Value Counts of Data Confidence in Old Radiators')
plt.xlabel('Data Confidence Levels')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


In [105]:
# Select only the specified columns
columns_to_keep = [
    "Section length", "Leg section depth", "Mid section depth", "Leg section height", "Mid section height",
    "Leg section weight", "Mid section weight","Heating surface", "Internal volume L", "Interaxis distance",
    "Nipple size - Top", "Nipple size - Bottom", "Output ΔT50"
]
test = test[columns_to_keep]

In [ ]:
print(test.dtypes)

In [ ]:
# number of unique values in each column
uniqueValues = test.nunique()

plt.figure(figsize=(10, 6))
ax = uniqueValues.plot(kind='bar', color='skyblue')
for patch in ax.patches: # count on top of each bar
    height= patch.get_height()
    width=patch.get_width()
    ax.text(patch.get_x() + width / 2,height,
            f'{int(patch.get_height())}', ha='center', va='bottom')

plt.title('Number of unique values in each column of Old Radiators')
plt.xlabel('Columns')
plt.ylabel('Unique value count')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
print(test.dtypes)

In [ ]:
#this shows how many heating surface values we have
# value counts of the 'Heating surface' column

heatingSurfaceCounts = test['Heating surface'].value_counts()
notAvailableCount = heatingSurfaceCounts.get('Not available')
print(notAvailableCount)

In [ ]:
df = test.drop(['Heating surface', 'Internal volume L'], axis=1, errors='ignore') #these 2 have a lot unique categorical values
print("Unique values in categorical columns:")
for col in df.select_dtypes(include=['object']).columns:
    print(df[col].value_counts())
    print('\n')

### Missing data except Heating Surface and Internal Volume L

In [ ]:
missingValues = test.isnull().sum()
plt.figure(figsize=(5, 5))
ax = missingValues.plot(kind='bar')

for i, value in enumerate(missingValues): #label of count on top
    ax.text(i, value + 0.01, str(value), ha='center', va='bottom')

plt.title('Number of missing values in each column of the Old Radiators')
plt.xlabel('Columns')
plt.ylabel('Number of Missing Values')
plt.xticks(rotation=90)
plt.show()

### Rename the columns indentical to train's

In [ ]:
# Rename columns
test.rename(columns={'Heating surface': 'Surface area', 'Internal volume L': 'Internal volume'}, inplace=True)

# Print to verify changes
print(test.columns)

In [ ]:
# Rename the column in test DataFrame
test.rename(columns={'Mid section depth': 'Mid section Depth'}, inplace=True)

print(" Column 'Mid section depth' renamed to 'Mid section Depth' in test data.")

In [114]:
test['Interaxis distance']=test['Interaxis distance'].fillna(test['Interaxis distance'].median())

### Make every columns into float

In [115]:
# convert to float
test['Nipple size - Top'] = test['Nipple size - Top'].apply(lambda x: float(x.replace('"', '')) if x != 0 else x)
test['Nipple size - Bottom'] = test['Nipple size - Bottom'].apply(lambda x: float(x.replace('"', '')) if x != 0 else x)

### Make the Nipple size criteria identical with the training set

In [116]:
test['Nipple size']=test['Nipple size - Bottom']
test=test.drop(columns=['Nipple size - Top','Nipple size - Bottom'])

In [ ]:
test.head()

### Missing data (Heating Surface and Internal volume)

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
import numpy as np

# Target variables
target_surface = 'Surface area'
target_volume = 'Internal volume'

# Features for modeling
features = ['Section length', 'Leg section depth', 'Mid section Depth', 'Leg section height',
            'Mid section height', 'Leg section weight', 'Mid section weight']

# Replace 'Not available' with NaN
test.replace('Not available', np.nan, inplace=True)


# Train model for predicting 'Heating surface'
test_surface_known = test.dropna(subset=features + [target_surface])
X_surface_known = test_surface_known[features]
y_surface_known = test_surface_known[target_surface]

imputer_surface = SimpleImputer(strategy="mean")
X_surface_known_imputed = imputer_surface.fit_transform(X_surface_known)

model_surface = LinearRegression()
model_surface.fit(X_surface_known_imputed, y_surface_known)

# Train model for predicting 'Internal volume L'
test_volume_known = test.dropna(subset=features + [target_volume])
X_volume_known = test_volume_known[features]
y_volume_known = test_volume_known[target_volume]

imputer_volume = SimpleImputer(strategy="mean")
X_volume_known_imputed = imputer_volume.fit_transform(X_volume_known)

model_volume = LinearRegression()
model_volume.fit(X_volume_known_imputed, y_volume_known)

# Predict missing values for 'Heating surface'
test_surface_missing = test[test[target_surface].isna()]
if not test_surface_missing.empty:
    X_surface_missing = imputer_surface.transform(test_surface_missing[features])
    test.loc[test_surface_missing.index, target_surface] = model_surface.predict(X_surface_missing)

# Predict missing values for 'Internal volume L'
test_volume_missing = test[test[target_volume].isna()]
if not test_volume_missing.empty:
    X_volume_missing = imputer_volume.transform(test_volume_missing[features])
    test.loc[test_volume_missing.index, target_volume] = model_volume.predict(X_volume_missing)

# Output message after processing
print("Linear Regression models trained and missing values filled.")

### Convert mm to dm to ensures consistency and prevents unit mismatches

In [119]:
test['Leg section depth'] = test['Leg section depth'] / 100
test['Mid section Depth'] = test['Mid section Depth'] / 100
test['Leg section height'] = test['Leg section height'] / 100
test['Mid section height'] = test['Mid section height'] / 100
test['Nipple size'] = test['Nipple size'] * 25.4 / 100

In [ ]:
test.head()

In [ ]:
df2 = test.drop(['Internal volume', 'Surface area'], axis=1) #correlation heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df2.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Matrix for Old Radiators")
plt.show()

In [ ]:
# Boxplot for 'Interaxis Distance' and 'Output ΔT50'
colList1 = ['Interaxis distance', 'Output ΔT50']
plt.figure(figsize=(6, 6))
sns.boxplot(data=test[colList1], palette='coolwarm')
plt.title('Boxplots of Interaxis Distance, Output ΔT50 in Old Radiators')
plt.xlabel('Columns')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.show()

# Boxplot for all other numeric columns excluding 'Interaxis Distance' and 'Output ΔT50'
plt.figure(figsize=(12, 8))
numericCols = test.select_dtypes(include=['number'])  # Select only numeric columns
toDrop = colList1
numericCols.drop(columns=toDrop).boxplot()
plt.suptitle("Boxplots of Other Numeric Columns in Old Radiators")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# pairplot
sns.pairplot(numericCols)
plt.suptitle('Pair Plots for All Numeric Columns in Test DataFrame', y=1.02)
plt.show()


In [ ]:
numPlots = len(numericCols.columns)
cols = 3
rows=np.ceil(numPlots / cols)   # calculating number of rows needed
rows = int(rows)

fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
axes = axes.flatten()

#histogram for each numeric column
for i, col in enumerate(numericCols.columns):
    axes[i].hist(numericCols[col], bins=20)
    sns.kdeplot(numericCols[col], ax=axes[i], color='red', linewidth=2)
    axes[i].set_title(f'Histogram and KDE of {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')

# hiding any unused subplots for better visualisation
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.suptitle('Histograms for Numeric Columns in Old Radiators', y=1.02)
plt.show()

In [ ]:
medianValues = numericCols.median()
modeValues = numericCols.mode().iloc[0]  # we take first mode if there are multiple
print("Median Values for Each Column:")
print(medianValues)
print('\n')
print("Mode Values for Each Column:")
print(modeValues)

meanValues = numericCols.mean()  #calculating means

plt.figure(figsize=(10, 6))
ax = meanValues.plot(kind='bar', color='skyblue')

for patch in ax.patches: #value labels on top of each bar
    height = patch.get_height()
    width=patch.get_width()
    ax.text(patch.get_x() + width/ 2, height + 0.05, f'{height:.2f}',
            ha='center', va='bottom')
plt.title('Mean of numeric columns in Old Radiators')
plt.xlabel('Columns')
plt.ylabel('Mean Value')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Modeling

In [126]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np

In [127]:
# defining features and targets using the complete set of features
features = [
    'Section length', 'Leg section depth', 'Mid section Depth',
    'Leg section height', 'Mid section height',
    'Leg section weight', 'Mid section weight',
    'Internal volume', 'Interaxis distance',
    'Nipple size', 'Surface area',
]

X = train[features]
y_k = train['Factor Km']
y_n = train['Exponent n']

# train/test split (80/20)
X_train, X_test, y_k_train, y_k_test = train_test_split(X, y_k, test_size=0.2, random_state=42)
y_n_train, y_n_test = train_test_split(y_n, test_size=0.2, random_state=42)

In [128]:
# scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [129]:
# defining RidgeCV model
alphas = np.logspace(-3, 3, 13)
ridge_k = RidgeCV(alphas=alphas, cv=5)
ridge_n = RidgeCV(alphas=alphas, cv=5)

In [ ]:
# 5-fold cross-validation on training set
cv = KFold(n_splits=5, shuffle=True, random_state=42)

ridge_scores_k = cross_val_score(ridge_k, X_train_scaled, y_k_train, cv=cv, scoring='neg_mean_squared_error')
ridge_scores_n = cross_val_score(ridge_n, X_train_scaled, y_n_train, cv=cv, scoring='neg_mean_squared_error')

ridge_cv_mse_k = -np.mean(ridge_scores_k)
ridge_cv_mse_n = -np.mean(ridge_scores_n)

print(f"[Ridge] Cross-Validation MSE (Factor Km): {ridge_cv_mse_k:.6f}")
print(f"[Ridge] Cross-Validation MSE (Exponent n): {ridge_cv_mse_n:.6f}")


In [ ]:
# fitting Ridge model on training set
ridge_k.fit(X_train_scaled, y_k_train)
ridge_n.fit(X_train_scaled, y_n_train)

In [132]:
# predict on test set
y_k_pred_ridge = ridge_k.predict(X_test_scaled)
y_n_pred_ridge = ridge_n.predict(X_test_scaled)

In [ ]:
# evaluating test set performance
ridge_mse_k = mean_squared_error(y_k_test, y_k_pred_ridge)
ridge_mse_n = mean_squared_error(y_n_test, y_n_pred_ridge)

ridge_r2_k = r2_score(y_k_test, y_k_pred_ridge)
ridge_r2_n = r2_score(y_n_test, y_n_pred_ridge)

print(f"[Ridge] Test MSE (Factor Km): {ridge_mse_k:.6f}")
print(f"[Ridge] Test MSE (Exponent n): {ridge_mse_n:.6f}")
print(f"[Ridge] R² Score (Factor Km): {ridge_r2_k:.4f}")
print(f"[Ridge] R² Score (Exponent n): {ridge_r2_n:.4f}")

In [134]:
from sklearn.ensemble import RandomForestRegressor

In [135]:
# defining models
rf_k = RandomForestRegressor(n_estimators=100, random_state=42)
rf_n = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:
# 5-fold cross-validation
rf_scores_k = cross_val_score(rf_k, X_train, y_k_train, cv=cv, scoring='neg_mean_squared_error')
rf_scores_n = cross_val_score(rf_n, X_train, y_n_train, cv=cv, scoring='neg_mean_squared_error')

rf_cv_mse_k = -np.mean(rf_scores_k)
rf_cv_mse_n = -np.mean(rf_scores_n)

print(f"[Random Forest] Cross-Validation MSE (Factor Km): {rf_cv_mse_k:.6f}")
print(f"[Random Forest] Cross-Validation MSE (Exponent n): {rf_cv_mse_n:.6f}")


In [ ]:
# fitting models on training set
rf_k.fit(X_train, y_k_train)
rf_n.fit(X_train, y_n_train)

In [ ]:
# predicting on test set
y_k_pred_rf = rf_k.predict(X_test)
y_n_pred_rf = rf_n.predict(X_test)

# evaluating on test set
rf_mse_k = mean_squared_error(y_k_test, y_k_pred_rf)
rf_mse_n = mean_squared_error(y_n_test, y_n_pred_rf)

rf_r2_k = r2_score(y_k_test, y_k_pred_rf)
rf_r2_n = r2_score(y_n_test, y_n_pred_rf)

print(f"[Random Forest] Test MSE (Factor Km): {rf_mse_k:.6f}")
print(f"[Random Forest] Test MSE (Exponent n): {rf_mse_n:.6f}")
print(f"[Random Forest] R² Score (Factor Km): {rf_r2_k:.4f}")
print(f"[Random Forest] R² Score (Exponent n): {rf_r2_n:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# feature importance
feature_names = X_train.columns

def plot_feature_importance_colored(importances, title):
    indices = np.argsort(importances)[::-1]
    sorted_names = feature_names[indices]
    sorted_importances = importances[indices]

    colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(sorted_importances)))

    plt.figure(figsize=(10, 5))
    plt.barh(sorted_names, sorted_importances, color=colors)
    plt.xlabel('Importance')
    plt.title(title)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

# plot for Factor Km and Exponent n
plot_feature_importance_colored(rf_k.feature_importances_, ' Random Forest Feature Importance — Factor Km')
plot_feature_importance_colored(rf_n.feature_importances_, ' Random Forest Feature Importance — Exponent n')

In [140]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# defining models
gb_k = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_n = GradientBoostingRegressor(n_estimators=100, random_state=42)

# cross-validation (5-fold)
gb_scores_k = cross_val_score(gb_k, X_train, y_k_train, cv=cv, scoring='neg_mean_squared_error')
gb_scores_n = cross_val_score(gb_n, X_train, y_n_train, cv=cv, scoring='neg_mean_squared_error')

gb_cv_mse_k = -np.mean(gb_scores_k)
gb_cv_mse_n = -np.mean(gb_scores_n)

print(f"[Gradient Boosting] Cross-Validation MSE (Factor Km): {gb_cv_mse_k:.6f}")
print(f"[Gradient Boosting] Cross-Validation MSE (Exponent n): {gb_cv_mse_n:.6f}")

In [ ]:
# fitting models
gb_k.fit(X_train, y_k_train)
gb_n.fit(X_train, y_n_train)


In [ ]:
# predicting on test set
y_k_pred_gb = gb_k.predict(X_test)
y_n_pred_gb = gb_n.predict(X_test)

# evaluating the test performance
gb_mse_k = mean_squared_error(y_k_test, y_k_pred_gb)
gb_mse_n = mean_squared_error(y_n_test, y_n_pred_gb)

gb_r2_k = r2_score(y_k_test, y_k_pred_gb)
gb_r2_n = r2_score(y_n_test, y_n_pred_gb)

print(f"[Gradient Boosting] Test MSE (Factor Km): {gb_mse_k:.6f}")
print(f"[Gradient Boosting] Test MSE (Exponent n): {gb_mse_n:.6f}")
print(f"[Gradient Boosting] R² Score (Factor Km): {gb_r2_k:.4f}")
print(f"[Gradient Boosting] R² Score (Exponent n): {gb_r2_n:.4f}")

In [ ]:
# feature importance gbr
feature_names = X_train.columns

def plot_feature_importance_colored(importances, title):
    indices = np.argsort(importances)[::-1]
    sorted_names = feature_names[indices]
    sorted_importances = importances[indices]
    colors = plt.cm.plasma(np.linspace(0.2, 0.9, len(sorted_importances)))

    plt.figure(figsize=(10, 5))
    plt.barh(sorted_names, sorted_importances, color=colors)
    plt.xlabel('Importance')
    plt.title(title)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

# Plot importances
plot_feature_importance_colored(gb_k.feature_importances_, ' Gradient Boosting Feature Importance — Factor Km')
plot_feature_importance_colored(gb_n.feature_importances_, ' Gradient Boosting Feature Importance — Exponent n')

In [ ]:
pip install xgboost

In [146]:
from xgboost import XGBRegressor

In [ ]:
# defining models for k and n
xgb_k = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
xgb_n = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# cross-validation (5-fold)
xgb_scores_k = cross_val_score(xgb_k, X_train, y_k_train, cv=cv, scoring='neg_mean_squared_error')
xgb_scores_n = cross_val_score(xgb_n, X_train, y_n_train, cv=cv, scoring='neg_mean_squared_error')

xgb_cv_mse_k = -np.mean(xgb_scores_k)
xgb_cv_mse_n = -np.mean(xgb_scores_n)

print(f"[XGBoost] Cross-Validation MSE (Factor Km): {xgb_cv_mse_k:.6f}")
print(f"[XGBoost] Cross-Validation MSE (Exponent n): {xgb_cv_mse_n:.6f}")

In [ ]:
# fitting models
xgb_k.fit(X_train, y_k_train)
xgb_n.fit(X_train, y_n_train)

In [ ]:
# predicting on test set
y_k_pred_xgb = xgb_k.predict(X_test)
y_n_pred_xgb = xgb_n.predict(X_test)

# evaluating on test set
xgb_mse_k = mean_squared_error(y_k_test, y_k_pred_xgb)
xgb_mse_n = mean_squared_error(y_n_test, y_n_pred_xgb)

xgb_r2_k = r2_score(y_k_test, y_k_pred_xgb)
xgb_r2_n = r2_score(y_n_test, y_n_pred_xgb)

print(f"[XGBoost] Test MSE (Factor Km): {xgb_mse_k:.6f}")
print(f"[XGBoost] Test MSE (Exponent n): {xgb_mse_n:.6f}")
print(f"[XGBoost] R² Score (Factor Km): {xgb_r2_k:.4f}")
print(f"[XGBoost] R² Score (Exponent n): {xgb_r2_n:.4f}")


In [ ]:
# feature importance plot
feature_names = X_train.columns

def plot_feature_importance_colored(importances, title):
    indices = np.argsort(importances)[::-1]
    sorted_names = feature_names[indices]
    sorted_importances = importances[indices]
    colors = plt.cm.cividis(np.linspace(0.2, 0.9, len(sorted_importances)))

    plt.figure(figsize=(10, 5))
    plt.barh(sorted_names, sorted_importances, color=colors)
    plt.xlabel('Importance')
    plt.title(title)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

plot_feature_importance_colored(xgb_k.feature_importances_, ' XGBoost Feature Importance — Factor Km')
plot_feature_importance_colored(xgb_n.feature_importances_, ' XGBoost Feature Importance — Exponent n')

In [151]:
model_names = ['Ridge', 'Random Forest', 'Gradient Boosting', 'XGBoost']

mse_k_values = [ridge_mse_k, rf_mse_k, gb_mse_k, xgb_mse_k]
mse_n_values = [ridge_mse_n, rf_mse_n, gb_mse_n, xgb_mse_n]

r2_k_values = [ridge_r2_k, rf_r2_k, gb_r2_k, xgb_r2_k]
r2_n_values = [ridge_r2_n, rf_r2_n, gb_r2_n, xgb_r2_n]


In [ ]:
# line plot for MSE
plt.figure(figsize=(10, 5))
plt.plot(model_names, mse_k_values, marker='o', label='Factor Km (MSE)')
plt.plot(model_names, mse_n_values, marker='s', label='Exponent n (MSE)')
plt.title('Model Comparison — Mean Squared Error')
plt.ylabel('MSE')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
# line plot for R²
plt.figure(figsize=(10, 5))
plt.plot(model_names, r2_k_values, marker='o', label='Factor Km (R²)')
plt.plot(model_names, r2_n_values, marker='s', label='Exponent n (R²)')
plt.title('Model Comparison — R² Score')
plt.ylabel('R²')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

# combining into a dataframe
comparison_df = pd.DataFrame({
    'Model': model_names,
    'MSE (Factor Km)': mse_k_values,
    'MSE (Exponent n)': mse_n_values,
    'R² (Factor Km)': r2_k_values,
    'R² (Exponent n)': r2_n_values
})
comparison_df = comparison_df.round(5)

# display the table
print(comparison_df)


In [ ]:
bar_width = 0.35
x = np.arange(len(model_names))

# plotting MSE for Factor Km vs Exponent n
plt.figure(figsize=(8, 5))
plt.bar(x - bar_width/2, mse_k_values, width=bar_width, label='Factor Km (MSE)', color='#4c72b0')
plt.bar(x + bar_width/2, mse_n_values, width=bar_width, label='Exponent n (MSE)', color='#dd8452')

plt.xticks(x, model_names)
plt.ylabel("MSE")
plt.title("Model Comparison — MSE")
plt.legend()
plt.grid(axis='y', linestyle=':', alpha=0.5)
plt.tight_layout()
plt.show()

# plotting R² for Factor Km vs Exponent n
plt.figure(figsize=(8, 5))
plt.bar(x - bar_width/2, r2_k_values, width=bar_width, label='Factor Km (R²)', color='#55a868')
plt.bar(x + bar_width/2, r2_n_values, width=bar_width, label='Exponent n (R²)', color='#c44e52')

plt.xticks(x, model_names)
plt.ylabel("R² Score")
plt.title("Model Comparison — R²")
plt.legend()
plt.grid(axis='y', linestyle=':', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
print(f"Train set size: {len(X_train)} rows")
print(f"Test set size: {len(X_test)} rows")

In [ ]:
def plot_predicted_vs_actual(y_true, y_pred, label):
    plt.figure(figsize=(6.5, 6.5))


    plt.scatter(
        y_true, y_pred,
        color="#4c72b0",
        edgecolor='white',
        linewidth=0.6,
        alpha=0.85,
        s=65
    )


    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    plt.plot([min_val, max_val], [min_val, max_val], linestyle='--', color='red', linewidth=1)
    plt.xlabel("Actual", fontsize=12)
    plt.ylabel("Predicted", fontsize=12)
    plt.title(f"{label}\nPredicted vs Actual", fontsize=13)
    plt.grid(True, linestyle=':', linewidth=0.6, alpha=0.5)
    plt.gca().set_facecolor("#f9f9f9")
    plt.tight_layout()
    plt.show()

# Plot for Factor Km
plot_predicted_vs_actual(y_k_test, y_k_pred_rf, "Random Forest — Factor Km")

# Plot for Exponent n
plot_predicted_vs_actual(y_n_test, y_n_pred_rf, "Random Forest — Exponent n")


In [ ]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

train_combined = train.copy()

train_combined['Average depth'] = (train_combined['Leg section depth'] + train_combined['Mid section Depth']) / 2
train_combined['Average height'] = (train_combined['Leg section height'] + train_combined['Mid section height']) / 2
train_combined['Average weight'] = (train_combined['Leg section weight'] + train_combined['Mid section weight']) / 2

columns_to_drop = [
    'Leg section depth', 'Mid section Depth',
    'Leg section height', 'Mid section height',
    'Leg section weight', 'Mid section weight'
]
train_combined = train_combined.drop(columns=columns_to_drop)

features_combined = [
    'Section length', 'Internal volume', 'Interaxis distance',
    'Nipple size', 'Surface area',
    'Average depth', 'Average height', 'Average weight'
]

X = train_combined[features_combined]
y_k = train_combined['Factor Km']
y_n = train_combined['Exponent n']

X_train, X_test, y_k_train, y_k_test = train_test_split(X, y_k, test_size=0.2, random_state=42)
_, _, y_n_train, y_n_test = train_test_split(X, y_n, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


models = {
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=500, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=500, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=5, random_state=42)
}

def print_cv_and_test_metrics(model, X_train, y_train, X_test, y_test, label, model_name):
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_mse_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
    cv_mse_mean = -np.mean(cv_mse_scores)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred)
    test_r2 = r2_score(y_test, y_pred)

    print(f"\n{model_name} — {label}:")
    print(f"  5-Fold CV MSE:  {cv_mse_mean:.6f}")
    print(f"  Test MSE:       {test_mse:.6f}")
    print(f"  Test R²:        {test_r2:.4f}")

for model_name, model in models.items():
    if model_name == 'Ridge':
        # Use scaled data
        print_cv_and_test_metrics(model, X_train_scaled, y_k_train, X_test_scaled, y_k_test, 'Factor Km', model_name)
        print_cv_and_test_metrics(model, X_train_scaled, y_n_train, X_test_scaled, y_n_test, 'Exponent n', model_name)
    else:
        # Use unscaled data
        print_cv_and_test_metrics(model, X_train, y_k_train, X_test, y_k_test, 'Factor Km', model_name)
        print_cv_and_test_metrics(model, X_train, y_n_train, X_test, y_n_test, 'Exponent n', model_name)

# Define feature importance plot function
def plot_feature_importance_colored(importances, feature_names, title):
    indices = np.argsort(importances)[::-1]
    sorted_names = feature_names[indices]
    sorted_importances = importances[indices]
    colors = plt.cm.cividis(np.linspace(0.2, 0.9, len(sorted_importances)))

    plt.figure(figsize=(10, 5))
    plt.barh(sorted_names, sorted_importances, color=colors, edgecolor='black', alpha=0.9)
    plt.xlabel('Importance')
    plt.title(title)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

rf_model_k = RandomForestRegressor(n_estimators=500, random_state=42)
rf_model_k.fit(X_train, y_k_train)
plot_feature_importance_colored(rf_model_k.feature_importances_, X_train.columns, 'Random Forest Feature Importance — Factor Km')

rf_model_n = RandomForestRegressor(n_estimators=500, random_state=42)
rf_model_n.fit(X_train, y_n_train)
plot_feature_importance_colored(rf_model_n.feature_importances_, X_train.columns, 'Random Forest Feature Importance — Exponent n')

# Hyperparameter Tuning

In [159]:
import numpy as np
import pandas as pd
from scipy.stats import randint, loguniform, uniform
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import warnings, pprint

warnings.filterwarnings("ignore", category=UserWarning)

In [160]:
# Reproducibility
np.random.seed(42)

In [161]:
# Cross‑validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    "rmse": "neg_root_mean_squared_error",
    "r2": "r2"
}

In [162]:
!pip install -U scikit-learn


In [ ]:
# Pipeline
ridge_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model",  Ridge(random_state=42))
])

param_dist_ridge = {
    "model__alpha":        loguniform(1, 100000),
    "model__fit_intercept": [True, False],
    "model__solver":      ["auto", "svd", "cholesky", "lsqr", "sag", "saga"],
}

# Factor Km
ridge_k = RandomizedSearchCV(
    estimator          = ridge_pipe,
    param_distributions= param_dist_ridge,
    n_iter             = 400,
    cv                 = cv,
    scoring            = scoring,
    refit              = "rmse",
    n_jobs             = -1,
    random_state       = 42,
    verbose            = 1
)
ridge_k.fit(X_train, y_k_train)

# Exponent n
ridge_n = RandomizedSearchCV(
    estimator          = ridge_pipe,
    param_distributions= param_dist_ridge,
    n_iter             = 400,
    cv                 = cv,
    scoring            = scoring,
    refit              = "rmse",
    n_jobs             = -1,
    random_state       = 42,
    verbose            = 1
)
ridge_n.fit(X_train, y_n_train)

# evaluate on hold-out set
def evaluate(label, model, X, y):
    pred  = model.predict(X)
    # rmse  = mean_squared_error(y, pred, squared=False)
    rmse = np.sqrt(mean_squared_error(y, pred))
    r2    = r2_score(y, pred)
    print(f"{label:<10} |  Test RMSE: {rmse:.5f}  |  R²: {r2:.4f}")

print("\nRidge Results:")
evaluate("Factor Km", ridge_k, X_test, y_k_test)
evaluate("Exponent n", ridge_n, X_test, y_n_test)

print("\nBest hyper-parameters:")
print("Factor Km :")
pprint.pprint(ridge_k.best_params_, indent=4, width=120)
print("\nExponent n:")
pprint.pprint(ridge_n.best_params_, indent=4, width=120)


In [ ]:
rf_pipe = Pipeline([("model", RandomForestRegressor(random_state=42, n_jobs=-1))])
rf_space = {
    "model__n_estimators": randint(200, 1000),
    "model__max_depth": randint(3, 31),
    "model__min_samples_split": randint(2, 11),
    "model__min_samples_leaf": randint(1, 11),
    "model__max_features": ["sqrt", "log2", None, 0.6, 0.8],
    "model__bootstrap": [True, False],
}

def rf_search(X_tr, y_tr, random_state):
    return RandomizedSearchCV(
        rf_pipe, rf_space, n_iter=200, cv=cv,
        scoring=scoring, refit="rmse",
        n_jobs=-1, random_state=random_state, verbose=1
    ).fit(X_tr, y_tr)

rf_k = rf_search(X_train, y_k_train, 42)
rf_n = rf_search(X_train, y_n_train, 42)

def eval_and_print(name, model, X_te, y_te):
    # rmse = mean_squared_error(y_te, model.predict(X_te), squared=False)
    rmse = np.sqrt(mean_squared_error(y_te, model.predict(X_te)))
    r2   = r2_score(y_te, model.predict(X_te))
    print(f"{name:<15} | test RMSE: {rmse:.5f} | R²: {r2:.4f}")

print("\nRandom Forest:")
eval_and_print("Factor Km", rf_k, X_test, y_k_test)
eval_and_print("Exponent n", rf_n, X_test, y_n_test)

print("\nRandom Forest best params — Km:")
pprint.pprint(rf_k.best_params_)
print("\nRandom Forest best params — n:")
pprint.pprint(rf_n.best_params_)


In [ ]:
gb_pipe = Pipeline([("model", GradientBoostingRegressor(random_state=42))])
gb_space = {
    "model__n_estimators": randint(50, 501),
    "model__learning_rate": loguniform(1e-3, 0.5),
    "model__max_depth": randint(2, 9),
    "model__subsample": uniform(0.5, 0.5),  # 0.5‑1.0
    "model__min_samples_split": randint(2, 11),
    "model__min_samples_leaf": randint(1, 11),
}

def gb_search(X_tr, y_tr, random_state):
    return RandomizedSearchCV(
        gb_pipe, gb_space, n_iter=100, cv=cv,
        scoring=scoring, refit="rmse",
        n_jobs=-1, random_state=random_state, verbose=1
    ).fit(X_tr, y_tr)

gb_k = gb_search(X_train, y_k_train, 42)
gb_n = gb_search(X_train, y_n_train, 42)

print("\nGradient Boosting")
eval_and_print("Factor Km", gb_k, X_test, y_k_test)
eval_and_print("Exponent n", gb_n, X_test, y_n_test)

print("\nGradient Boosting best params — Km:")
pprint.pprint(gb_k.best_params_)
print("\nGradient Boosting best params — n:")
pprint.pprint(gb_n.best_params_)

In [ ]:
from xgboost import XGBRegressor

xgb_pipe = Pipeline([
    ("model", XGBRegressor(
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1,
        tree_method="hist"
    ))
])

xgb_space = {
    "model__n_estimators": randint(50, 701),
    "model__learning_rate": loguniform(1e-3, 1.0),
    "model__max_depth": randint(2, 11),
    "model__subsample": uniform(0.5, 0.5),
    "model__colsample_bytree": uniform(0.5, 0.5),
    "model__gamma": [0, 0.1, 0.2, 0.5, 1],
    "model__reg_lambda": loguniform(1e-3, 100),
    "model__reg_alpha":  loguniform(1e-3, 10),
}

def xgb_search(X_tr, y_tr, random_state):
    return RandomizedSearchCV(
        xgb_pipe, xgb_space, n_iter=400, cv=cv,
        scoring=scoring, refit="rmse",
        n_jobs=-1, random_state=random_state, verbose=1
    ).fit(X_tr, y_tr)

xgb_k = xgb_search(X_train, y_k_train, 42)
xgb_n = xgb_search(X_train, y_n_train, 42)

print("\nXGBoost")
eval_and_print("Factor Km", xgb_k, X_test, y_k_test)
eval_and_print("Exponent n", xgb_n, X_test, y_n_test)

print("\nXGBoost best params — Km:")
pprint.pprint(xgb_k.best_params_)
print("\nXGBoost best params — n:")
pprint.pprint(xgb_n.best_params_)

In [ ]:
print(test)

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Defining features and targets
features = [
    'Section length', 'Leg section depth', 'Mid section Depth',
    'Leg section height', 'Mid section height',
    'Leg section weight', 'Mid section weight',
    'Internal volume', 'Interaxis distance',
    'Nipple size', 'Surface area'
]

X = train[features]
y_k = train['Factor Km']
y_n = train['Exponent n']

# Making sure test data has matching columns
X_test_features = test[features]

# Best hyperparameters from tuning
best_params_k = {
    'bootstrap': True,
    'max_depth': 16,
    'max_features': 'sqrt',
    'min_samples_leaf': 1,
    'min_samples_split': 4,
    'n_estimators': 884,
    'random_state': 42
}

best_params_n = {
    'bootstrap': False,
    'max_depth': 9,
    'max_features': 'sqrt',
    'min_samples_leaf': 1,
    'min_samples_split': 8,
    'n_estimators': 285,
    'random_state': 42
}

# Training final models on all data
final_rf_k = RandomForestRegressor(**best_params_k)
final_rf_n = RandomForestRegressor(**best_params_n)

final_rf_k.fit(X, y_k)
final_rf_n.fit(X, y_n)

print("Final Random Forest models trained on full dataset!")

In [174]:
# Predicting on test dataset
y_k_test_pred = final_rf_k.predict(X_test_features)
y_n_test_pred = final_rf_n.predict(X_test_features)

# Adding predictions to test dataframe
test['Predicted Factor Km'] = y_k_test_pred
test['Predicted Exponent n'] = y_n_test_pred

# Saving predictions to Excel
output_path = '/Your_path/test_predictions.xlsx'
test.to_excel(output_path, index=False)

print(f"Predictions saved to: {output_path}")

In [ ]:
# Getting feature importance for Factor Km model
importance_k = pd.Series(final_rf_k.feature_importances_, index=features).sort_values(ascending=False)
importance_n = pd.Series(final_rf_n.feature_importances_, index=features).sort_values(ascending=False)

print("\nFeature Importance (Factor Km):")
print(importance_k)

print("\nFeature Importance (Exponent n):")
print(importance_n)

# Plotting feature importance
def plot_feature_importance(importances, title):
    plt.figure(figsize=(8, 5))
    importances.plot(kind='barh', color='#4c72b0')
    plt.xlabel('Importance Score')
    plt.title(title)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

plot_feature_importance(importance_k, 'Feature Importance — Factor Km')
plot_feature_importance(importance_n, 'Feature Importance — Exponent n')


In [ ]:
feature_names = X.columns

def plot_feature_importance_colored_viridis(importances, feature_names, title):
    indices = np.argsort(importances)[::-1]
    sorted_names = feature_names[indices]
    sorted_importances = importances[indices]
    colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(sorted_importances)))

    plt.figure(figsize=(10, 5))
    plt.barh(sorted_names, sorted_importances, color=colors, edgecolor='black', alpha=0.9)
    plt.xlabel('Importance', fontsize=12)
    plt.title(title, fontsize=14, pad=10)
    plt.gca().invert_yaxis()
    plt.grid(axis='x', linestyle=':', alpha=0.5)
    plt.tight_layout()
    plt.show()

# Plot for Factor Km
plot_feature_importance_colored_viridis(final_rf_k.feature_importances_, feature_names, 'Random Forest Feature Importance — Factor Km')

# Plot for Exponent n
plot_feature_importance_colored_viridis(final_rf_n.feature_importances_, feature_names, 'Random Forest Feature Importance — Exponent n')


In [ ]:
# Calculate mean of predicted Factor Km and Exponent n
mean_k = test['Predicted Factor Km'].mean()
mean_n = test['Predicted Exponent n'].mean()

print(f"Mean Predicted Factor Km: {mean_k:.4f}")
print(f"Mean Predicted Exponent n: {mean_n:.4f}")

In [ ]:
# Get descriptive statistics for predictions
stats = test[['Predicted Factor Km', 'Predicted Exponent n']].describe()

print(stats)